# 勾配降下法の収束

参考
* [Gradient Descent: Convergence Analysis](https://www.stat.cmu.edu/~ryantibs/convexopt-F13/scribes/lec6.pdf)
* [非凸最適化アルゴリズムとその計算量解析](https://www.ism.ac.jp/~mirai/sscoke/2024/marumo-lecture.pdf)

---

関数$f$は凸であり，微分可能であるとします．また，Lipschitz連続であり，$\|\nabla f(x)-\nabla f(y)\|_2 \leq L\|x-y\|_2$を満たす$L$が存在するとします．このとき，勾配降下法をステップサイズ$t = 1 / L$で$k$回実行した時，得られる解は

$$
f\left(x^{(k)}\right)-f\left(x^*\right) \leq \frac{L \left\|x^{(0)}-x^*\right\|_2^2}{2k}
$$

を満たします．

---

## 勾配降下法の収束の証明

---

**解析の定石**
一般に，勾配法で解の列$(x_k)_k$が得られたとき，次の２つを示すことで収束を示していきます：

1. $\left\|x_{k+1}-x_k\right\|$が大きいとき → 勾配更新によって目的関数が十分下がる
2. $\left\|x_{k+1}-x_k\right\|$が小さい時 → 勾配のノルムが小さい

を示し，どちらに転んでも嬉しい状況を作ります．

---


### 平滑性を利用して目的関数を上から抑える

**上界補題**

$\nabla f$がLipschitz連続なので，$\nabla^2 f(x) \preceq L I$もしくは$\nabla^2 f(x)-L I$が半負定値行列に成ります．これを使って$f$を二次の項まで分解すると，

$$
\begin{aligned}
f(y) & \leq f(x)+\nabla f(x)^T(y-x)+\frac{1}{2} \nabla^2 f(x)\|y-x\|_2^2 \\
& \leq f(x)+\nabla f(x)^T(y-x)+\frac{1}{2} L\|y-x\|_2^2
\end{aligned}
$$

が得られます．

In [18]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact

# 適当な関数（非凸）を用意します
xs = np.linspace(-0.6, 0.5, 30)

def f(x):
    return x ** 4 - 0.7 * x **2 + 0.1 * x + 1.5 * np.cos(10 * x) * 0.1 + 0.3

def f_grad(x):
    grad = 4 * x ** 3 - 1.4 * x + 0.1 - 1.5 * np.sin(10 * x)
    return grad

L = np.max(np.abs(f_grad(xs))) * 10

def plot_upper_bound(y):
    up = f(y) + f_grad(y) * (xs - y) + L / 2 * (xs - y) ** 2
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 1, 1)
    plt.plot(xs, f(xs), label=r"$f$")
    plt.plot(xs, up, label=r"upper")

    plt.grid()
    plt.legend()
    plt.ylim(-0.8, 0.8)
    plt.xlabel("x")

    plt.show()

interact(plot_upper_bound, y=(-0.6, 0.5, 0.1))

interactive(children=(FloatSlider(value=-0.09999999999999998, description='y', max=0.5, min=-0.6), Output()), …

<function __main__.plot_upper_bound(y)>

### 停留点を出す

更新式を思い出すと，$\nabla f(x) = L(x - x^+)$なので，上界補題より，

$$
f\left(x^{+}\right) \leq f(x)+\nabla f(x)^T\left(x^{+}-x\right)+\frac{1}{2} L\left\|x^{+}-x\right\|_2^2
 = - \frac{1}{2} L\left\|x^{+}-x\right\|_2^2
$$
なので，

1. $f\left(x^{+}\right) - f(x) \leq -\frac{L}{2} \|x^+ - x\|_2^2$を得ます．
    * すなわち，**更新時の差が大きい場合は，関数値の減少量が大きい**ことが言えます．
2. 一方で，$\frac{1}{2L}\|\nabla f(x)\|^2 = \frac{L}{2} \|x^+ - x\|^2$もなりたちます．
    * つまり，**更新時の差が小さい場合は，勾配ノルムが小さい**ことも言えます．

よって，この２つを組み合わせると，
$$
\frac{1}{2L} \|\nabla f(x_k)\|^2 \leq f(x_k) - f(x_{k+1})
$$
なので，$k$について足し合わせると，
$$
\min_{0 \leq i < k} \|\nabla f(x_i)\|^2
\leq \frac{1}{k} \sum^{k-1}_{i=0} \|\nabla f(x_i)\|^2
\leq \frac{2L}{k}\underbrace{\left(f(x_0) - \inf_{x \in \mathbb{R}^d} f(x)\right)}_{\leq \Delta}
$$

が成り立ちます．まとめると，
$$
\min_{0 \leq i < k} \|\nabla f(x_i)\|
\leq \sqrt{\frac{2L \Delta}{k}}
$$
が得られ，停留点に$1/\sqrt{k}$で落ちていくことがわかります．

### 停留点と最適解を関連させる

* [The Polyak-Lojasiewicz (PL) Condition](https://www.stat.cmu.edu/~siva/teaching/725/lec13.pdf)

強凸な場合，後は停留点と最適解はPL条件で関連付けられます．

---

**補題** $\alpha$-強凸な関数は，$\alpha$-PL条件を満たします．

次の$\alpha$-強凸関数を考えましょう：
$$
\begin{aligned}
f\left(x^*\right) 
&\geq f(x)+\nabla f(x)^t\left(x^*-x\right) + \frac{\alpha}{2}\|x - x^*\|^2\\
&\geq f(x) - \frac{1}{2\alpha}\|\nabla f(x)\|^2
\end{aligned}
$$
です．ここで，二行目は右辺を$x^*$が変数の二次方程式として考えて，それを解いてます．
よって，明らかに
$$
f(x) - f\left(x^*\right) 
\leq \frac{1}{2\alpha}\|\nabla f(x)\|^2
$$
が成り立つので，停留点に近づくと最適解に近くなることも言えます．


### 補足：一般の凸のケース

こっちのほうがよく出てきますが，上の方がすぐに出せるので僕は好き（強凸性使うけど）

関数が凸なとき，最適解$x^*$について，

$$
\begin{aligned}
& f\left(x^*\right) \geq f(x)+\nabla f(x)^T\left(x^*-x\right) \\
\Rightarrow& f(x) - f\left(x^*\right)\leq \nabla f(x)^T\left(x-x^*\right)
\end{aligned}
$$

が成り立ちます．
$$
\frac{1}{2L} \|\nabla f(x_k)\|^2 \leq f(x_k) - f(x_{k+1})
$$
を使って変形すると（$t= 1 / L$），

$$
\begin{aligned}
& f\left(x^{+}\right) \leq f\left(x^*\right)+\nabla f(x)^T\left(x-x^*\right)-\frac{t}{2}\|\nabla f(x)\|_2^2 \\
& f\left(x^{+}\right)-f\left(x^*\right) \leq \frac{1}{2 t}\left(2 t \nabla f(x)^T\left(x-x^*\right)-t^2\|\nabla f(x)\|_2^2\right) \\
& f\left(x^{+}\right)-f\left(x^*\right) \leq \frac{1}{2 t}\left(2 t \nabla f(x)^T\left(x-x^*\right)-t^2\|\nabla f(x)\|_2^2-\left\|x-x^*\right\|_2^2+\left\|x-x^*\right\|_2^2\right) \\
& f\left(x^{+}\right)-f\left(x^*\right) \leq \frac{1}{2 t}\left(\left\|x-x^*\right\|_2^2-\left\|x-t \nabla f(x)-x^*\right\|_2^2\right)
\end{aligned}
$$

です．$x^{+}=x-t \nabla f(x)$なので，

$$
f\left(x^{+}\right)-f\left(x^*\right) \leq \frac{1}{2 t}\left(\left\|x-x^*\right\|_2^2-\left\|x^{+}-x^*\right\|_2^2\right)
$$

が成り立ちます．イテレーション全体で足し合わせば，

$$
\begin{aligned}
\sum_{i=1}^k f\left(x^{(i)}-f\left(x^*\right)\right. & \leq \sum_{i=1}^k \frac{1}{2 t}\left(\left\|x^{(i-1)}-x^*\right\|_2^2-\left\|x^{(i)}-x^*\right\|_2^2\right) \\
& =\frac{1}{2 t}\left(\left\|x^{(0)}-x^*\right\|_2^2-\left\|x^{(k)}-x^*\right\|_2^2\right) \\
& \leq \frac{1}{2 t}\left(\left\|x^{(0)}-x^*\right\|_2^2\right)
\end{aligned}
$$

であり，$x^{(k)}$は毎回必ず$x^*$に近づくので，

$$
\begin{aligned}
f\left(x^{(k)}\right)-f\left(x^*\right) & \leq \frac{1}{k} \sum_{i=1}^k f\left(x^{(i)}\right)-f\left(x^*\right) \\
& \leq \frac{\left\|x^{(0)}-x^*\right\|_2^2}{2 t k}
\end{aligned}
$$

を得ます．

<!-- ## 強凸関数での勾配のノルムのバウンド

全体的に間違い

参考:
* [Beck本](https://epubs.siam.org/doi/10.1137/1.9781611974997)のstrong convexityの応用（Theorem 5.25など）

[劣勾配法の解析](OPT_Grad_subgradient.ipynb)などを使えば，$f(x^{(k)}) - f(x^*)$を上からバウンドすることができます．
さらに，$f$が特に強凸関数ならば，smoothnessの係数を出さなくても勾配について収束を言うことができます．

準備:
* $f$を微分可能な強凸関数
* 射影勾配法：$x^{k+1} = \arg\min_{x \in \mathcal{C}} f(x^k) + \langle \nabla f(x^k), x - x^k\rangle + \frac{L}{2} \|x - x^k\|^2$
    * これは$x^{k+1} = \arg\min_{x \in \mathbb{R}^d} f(x^k) + \langle \nabla f(x^k), x - x^k\rangle + \mathbb{I}_C(x) + \frac{L}{2} \|x - x^k\|^2$
    * これは$x^{k+1} = \operatorname{Proj}_{\mathcal{C}} (x^k - \frac{1}{L} \nabla f(x^k))$と同じです．
    * $T_L(x^k) = x^{k+1}$とします． 
    * また，$\psi(u) = f(y) + \langle \nabla f(y), u - y\rangle + \mathbb{I}_\mathcal{C}(u) + \frac{L}{2} \|u - y\|^2$とします．あきらかに，$T_L(y) = \arg\min_{u \in \mathbb{R}^d} \psi(u)$です．

$\varepsilon \geq f(x) - f(x^*)$を満たす$\varepsilon$最適な$x$を考えましょう．このとき，$f$は強凸なので
$\psi(u)$は$u$について$L$-強凸なので，
$$\psi(x) - \psi(T_L(y)) \geq \frac{L}{2}\|x - T_L(y)\|^2$$
が成立します（証明はBeck本Theorem 5.25）． -->